In [7]:
import cv2
import torch
from pathlib import Path
from deepface import DeepFace
import numpy as np

# Define the device to run the model on
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the YOLOv7 model
model_path = './yolov7/yolov7.pt'  # pastikan file yolov7.pt ada di direktori kerja Anda
model = torch.load(model_path, map_location=device)['model']
model = model.half().to(device)
model.eval()

# Function to perform inference
def detect_faces(img, model):
    if img is None:
        raise ValueError("Image not loaded properly")
    
    # Prepare the image
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img_rgb, (640, 640))  # Resize to the input size expected by YOLOv7
    img_tensor = torch.from_numpy(img_resized).permute(2, 0, 1).unsqueeze(0).half().to(device)
    img_tensor /= 255.0  # normalize to [0, 1]
    
    if img_tensor is None:
        raise ValueError("Tensor conversion failed")
    
    # Debug: Print tensor shape
    print(f"Tensor shape: {img_tensor.shape}")

    # Run inference
    with torch.no_grad():
        pred = model(img_tensor)[0]

    # Process predictions
    pred = pred[pred[:, 4] > 0.5]  # threshold confidence
    boxes = pred[:, :4].cpu().numpy()
    return boxes

# Load image
image_path = './tests/freya.jpg'
img = cv2.imread(image_path)

# Perform face detection using YOLOv7
try:
    boxes = detect_faces(img, model)
except Exception as e:
    print(f"Error during face detection: {e}")
    boxes = []

# Loop through detected faces and classify gender, estimate age
for box in boxes:
    x1, y1, x2, y2 = box.astype(int)
    face = img[y1:y2, x1:x2]

    # Use DeepFace to analyze the face
    analysis = DeepFace.analyze(face, actions=['age', 'gender'])

    age = analysis['age']
    gender = analysis['gender']

    # Draw bounding box and label on the image
    label = f"{gender}, {age}"
    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

# Display the image
cv2.imshow('Image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()


Tensor shape: torch.Size([1, 3, 640, 640])
Error during face detection: The shape of the mask [1, 85] at index 1 does not match the shape of the indexed tensor [1, 25200, 85] at index 1


: 